<a href="https://colab.research.google.com/github/EuclidStellar/RnD_on_code_review_methods/blob/main/R%26D_on_code_review_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install colorama

In [19]:
import difflib
from colorama import Fore, Style, init
from IPython.display import display, HTML

# Initialize colorama
init(autoreset=True)

def get_diff(file1, file2):
    """
    Compares two text files line by line and returns differences with colored additions and reductions.
    The output will include line numbers for changes and will be displayed in a pleasing format.
    """
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

    # Get the diff as a list of lines
    diff = list(difflib.unified_diff(lines1, lines2, lineterm=''))

    # Variables to track the line numbers
    old_line_number = 1
    new_line_number = 1

    # HTML for the output
    html_diff = ""

    # Iterate through the diff and colorize it
    for line in diff:
        # If it's a change marker (e.g., line chunk header)
        if line.startswith('@@'):
            # Get the old and new line numbers from the chunk header
            chunk_info = line.split(" ")[1:3]
            old_start_line = int(chunk_info[0].split(",")[0][1:])
            new_start_line = int(chunk_info[1].split(",")[0][1:])
            html_diff += f'<div style="color: #FFA500; font-weight: bold; margin-top: 10px;">Changes start at lines {old_start_line} and {new_start_line}</div>'
        elif line.startswith('-'):  # Lines removed
            html_diff += f'<div style="color: red;">Line {old_line_number}: {line[1:].strip()}</div>'
            old_line_number += 1
        elif line.startswith('+'):  # Lines added
            html_diff += f'<div style="color: green;">Line {new_line_number}: {line[1:].strip()}</div>'
            new_line_number += 1
        else:  # Unchanged lines
            old_line_number += 1
            new_line_number += 1

    # If there are no changes, display a friendly message
    if not html_diff:
        html_diff = "<div style='color: green; font-weight: bold;'>No changes detected.</div>"

    # Display the HTML in Colab
    return html_diff

# Example usage
file1 = "/content/new_code.py"  # Replace with the correct path to your file
file2 = "/content/old_code.py"  # Replace with the correct path to your file

# Get the colored diff
colored_output = get_diff(file1, file2)

# Display the colored diff in Colab
display(HTML(f"<div style='font-family: Arial, sans-serif; padding: 10px; font-size: 14px;'>{colored_output}</div>"))


In [31]:
import re
from colorama import Fore, Style, init

# Initialize colorama for colored output
init(autoreset=True)

def check_unclosed_brackets(code):
    """
    Checks if all brackets are properly closed.
    Supports: (), {}, []
    Returns which brackets are unclosed and at which line number.
    """
    stack = []  # Stack to hold the opening brackets and their line numbers
    brackets = {'(': ')', '{': '}', '[': ']'}  # Mapping for opening and closing brackets
    opening_brackets = set(brackets.keys())  # Set of opening brackets
    closing_brackets = set(brackets.values())  # Set of closing brackets
    line_number = 1
    lines = code.split('\n')

    # List to hold error messages
    result = []

    # Iterate through each line and each character in the line
    for line in lines:
        for char in line:
            if char in opening_brackets:
                # Push opening bracket with the current line number onto the stack
                stack.append((char, line_number))
            elif char in closing_brackets:
                if not stack:
                    # If stack is empty but we encounter a closing bracket, error!
                    result.append(Fore.RED + f"Brackets are not balanced! Error at line {line_number}, unexpected closing bracket '{char}'.")
                    return "\n".join(result)

                # Pop the top item from the stack and check if it matches the current closing bracket
                last_opening, last_line = stack.pop()
                if brackets[last_opening] != char:
                    # If the last opening bracket doesn't match the current closing bracket, it's an error
                    result.append(Fore.RED + f"Brackets are not balanced! Error at line {line_number}, expected closing for '{last_opening}', found '{char}' instead.")
                    return "\n".join(result)

        line_number += 1

    # After the loop, if the stack still contains any opening brackets, report them as unbalanced
    if stack:
        for unclosed_bracket, unclosed_line in stack:
            result.append(Fore.RED + f"Brackets are not balanced! The opening '{unclosed_bracket}' at line {unclosed_line} is not closed.")

    # If no issues, all brackets are balanced
    if not result:
        result.append(Fore.GREEN + "Brackets are balanced.")

    return "\n".join(result)

# Example usage with a large number of unbalanced brackets
code_sample = """
def check_brackets():
    if (x > 0 {
        print("Hello")
    if (y < 5 {
        for i in range(10):
            if (i % 2 == 0) {
                print(i)
            }
    else:
        print("Done")

def main():
    def nested_function():
        a = (2 + 3)
        if (a > 5):
            print(a
        else:
            print("Error")
    return
"""

# Test the function
print(check_unclosed_brackets(code_sample))


Brackets are not balanced! The opening '(' at line 3 is not closed.
Brackets are not balanced! The opening '{' at line 3 is not closed.
Brackets are not balanced! The opening '(' at line 5 is not closed.
Brackets are not balanced! The opening '{' at line 5 is not closed.
Brackets are not balanced! The opening '(' at line 17 is not closed.


In [36]:
import ast

def check_unclosed_brackets_ast(filename):
    """
    Checks if all brackets ({, }, (, ), [, ]) are properly closed in Python code
    by parsing the code using Python's Abstract Syntax Tree (AST) module.
    It returns the line number and type of unbalanced brackets.
    """
    with open(filename, "r") as file:
        code = file.read()

    try:
        # Attempt to parse the code using AST (this will catch syntax errors)
        ast.parse(code)

        return f"Brackets are balanced in {filename}."

    except SyntaxError as e:
        # SyntaxError contains the line number where parsing failed
        error_message = str(e)
        line_number = e.lineno
        column_number = e.offset
        msg = f"Brackets are not balanced in {filename}.\n"
        msg += f"Error at line {line_number}, column {column_number}. {error_message}"

        return msg


# Example usage
print(check_unclosed_brackets_ast("ast_code.py"))


Brackets are not balanced in ast_code.py.
Error at line 2, column 15. invalid syntax (<unknown>, line 2)


In [37]:
!pip install requests  # For API requests


In [51]:
import requests
from IPython.display import display, Markdown

def review_code_with_gemini(api_key, file_path):
    """
    Reviews the code in the given file using Gemini's API and provides feedback.
    It returns a more user-friendly response with feedback and potential issues in the code.
    """
    # Read the content of the code file
    try:
        with open(file_path, 'r') as file:
            code_to_review = file.read()
    except FileNotFoundError:
        return f"Error: The file '{file_path}' was not found."

    # Gemini API URL (Make sure this URL is correct for the Gemini model you're using)
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json"
    }

    # Improved prompt with file content for review
    prompt_text = f"""
    Please review the following Python code snippet. Your task is to provide feedback on first provide feedback:
    1. Syntax errors
    2. Logical errors
    3. Missing or incorrect return statements in functions.
    4. Code structure issues
    5. Potential performance issues or optimizations.
    6. Best practices the code may not follow
    Do not give me feedback more than 15 lines

    Here is the code to review:

    {code_to_review}
    """

    payload = {
        "contents": [{
            "parts": [{"text": prompt_text}]
        }]
    }

    # Make the POST request to the Gemini API
    response = requests.post(url, headers=headers, json=payload)

    # Handle response and format it for better readability
    if response.status_code == 200:
        gemini_response = response.json()
        # Check if candidates exist in the response
        if 'candidates' in gemini_response and len(gemini_response['candidates']) > 0:
            feedback = gemini_response['candidates'][0]['content']['parts'][0]['text']
            return format_gemini_response(feedback)
        else:
            return "Error: Gemini did not return valid feedback. The response structure may have changed."
    else:
        return f"Error: {response.status_code}, {response.text}"

def format_gemini_response(response_text):
    """
    Format the Gemini response into a more readable and useful format with Markdown support.
    """
    # Add markdown formatting (code blocks and sections)
    formatted_response = f"### Code Review Feedback\n\n{response_text.strip()}"

    # Ensure the feedback text has appropriate Markdown syntax for code blocks
   # formatted_response = formatted_response.replace("```python", "```python\n").replace("\n", "\n\n")

    # Display the formatted response as Markdown
    display(Markdown(formatted_response))


# Example usage
api_key = "your-API-KEY"  # Use your actual API key here
file_path = "/content/ast_code.py"  # Replace with the path to your code file

# Get the Gemini review response and display it nicely
review_response = review_code_with_gemini(api_key, file_path)
print(review_response)


### Code Review Feedback

Here's a review of the Python code snippet:

1.  **Syntax Errors:** Missing colons after `if`, `else`, and `for` statements. Incorrect use of curly braces `{}` instead of colons `:` for code blocks. Syntax error: `print(a` missing closing parenthesis.
2.  **Logical Errors:** `x` and `y` are not defined, leading to `NameError`. The `else` block associated with the `y < 5` conditional is misplaced and will always execute.
3.  **Missing/Incorrect Returns:** `check_brackets` implicitly returns `None`. `main` returns nothing, which is acceptable, but could be improved.
4.  **Code Structure:** `check_brackets` lacks clear purpose and function name is misapplied (no bracket checking). `nested_function` defined but never called within `main`.
5.  **Performance:** No significant performance issues in this small snippet.
6.  **Best Practices:** Indentation is inconsistent and incorrect. Variable names should be descriptive. Avoid defining functions inside other functions unless necessary for specific use cases like closures or decorators. Avoid redundant parenthesis.

None
